In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt

Data for rides were downloaded from: https://touringplans.com/walt-disney-world/crowd-calendar#DataSets

In [2]:
meta=pd.read_csv('metadata.csv')
display(meta)
# Select only the columns that are necessary
meta = meta[['DATE','DAYOFWEEK','DAYOFYEAR','WEEK','MONTH','YEAR','SEASON']]
meta['DATE']=pd.to_datetime(meta['DATE'])
# Add a day of the month column
meta['DAYOFMONTH']=meta.DATE.dt.day
display(meta)

,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEK,MONTH,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,...,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN
0,1/1/2015,NaN,5,0,0,1,2015,CHRISTMAS PEAK,0,5,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,1/2/2015,NaN,6,1,0,1,2015,CHRISTMAS,2,5,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,1/3/2015,NaN,7,2,0,1,2015,CHRISTMAS,3,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,1/4/2015,NaN,1,3,1,1,2015,CHRISTMAS,4,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,1/5/2015,NaN,2,4,1,1,2015,CHRISTMAS,5,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2074,8/27/2021,NaN,6,238,34,8,2021,NaN,11,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2075,8/28/2021,NaN,7,239,34,8,2021,NaN,10,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2076,8/29/2021,NaN,1,240,35,8,2021,NaN,9,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2077,8/30/2021,NaN,2,241,35,8,2021,NaN,8,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


,DATE,DAYOFWEEK,DAYOFYEAR,WEEK,MONTH,YEAR,SEASON,DAYOFMONTH
0,2015-01-01,5,0,0,1,2015,CHRISTMAS PEAK,1
1,2015-01-02,6,1,0,1,2015,CHRISTMAS,2
2,2015-01-03,7,2,0,1,2015,CHRISTMAS,3
3,2015-01-04,1,3,1,1,2015,CHRISTMAS,4
4,2015-01-05,2,4,1,1,2015,CHRISTMAS,5
...,...,...,...,...,...,...,...,...
2074,2021-08-27,6,238,34,8,2021,NaN,27
2075,2021-08-28,7,239,34,8,2021,NaN,28
2076,2021-08-29,1,240,35,8,2021,NaN,29
2077,2021-08-30,2,241,35,8,2021,NaN,30


In [3]:
folders=os.listdir('./Rides/')
paths=[]
parks=[]
rides=[]
for f in folders:
    parks_list=os.listdir('./Rides/'+f)
    for i,r in enumerate(parks_list):
        path='./Rides/'+f+'/'+r
        paths.append(path)
        rides.append(r[:-4])
        parks.append(f)
print(np.array(paths))
print(np.array(rides))
print(parks)

['./Rides/Animal_Kingdom/dinosaur.csv'
 './Rides/Animal_Kingdom/expedition_everest.csv'
 './Rides/Animal_Kingdom/flight_of_passage.csv'
 './Rides/Animal_Kingdom/kilimanjaro_safaris.csv'
 './Rides/Animal_Kingdom/navi_river.csv' './Rides/Epcot/soarin.csv'
 './Rides/Epcot/spaceship_earth.csv']
['dinosaur' 'expedition_everest' 'flight_of_passage' 'kilimanjaro_safaris'
 'navi_river' 'soarin' 'spaceship_earth']
['Animal_Kingdom', 'Animal_Kingdom', 'Animal_Kingdom', 'Animal_Kingdom', 'Animal_Kingdom', 'Epcot', 'Epcot']


In [4]:
def clean_df(path,park):
    
    df1=pd.read_csv(path)
    
    # Make datetime a datetime object and round times to 5min
    df1['datetime']=pd.to_datetime(df1['datetime']).dt.round("5min")
    
    # There are some values at -999, replace these with nans
    df1[df1['SPOSTMIN']<0]=np.nan 
    
    #restrict to 2015-2018 so we dont have a huge file that takes up too much memory and 
    # because 2020-2022 were not normal years
    df1=df1[(df1['datetime'].dt.year<=2018) & (df1['datetime'].dt.year>=2015)]
    
    # Rename date to DATE and make it a datetime object
    df1=df1.rename(columns={"date":"DATE"})
    df1['DATE']=pd.to_datetime(df1['DATE'])

    # Combine the SPOSTMIN and SACTMIN into one column (looks like when one is nan, the other isn't)
    df1['SPOSTMIN'].update(df1.pop('SACTMIN'))

    # Rename SPOSTMIN to wait for wait time
    df1=df1.rename(columns={'SPOSTMIN':'wait'})
    
    # Add the park name
    df1['park']=park
    
    display(df1)
    
    return df1

In [5]:
# Select one ride from Animal Kingdom and one from Epcot
df1=clean_df(paths[0],parks[0])
df2=clean_df(paths[-1],parks[-1])

,DATE,datetime,wait,park
0,2015-01-01,2015-01-01 07:45:00,5.0,Animal_Kingdom
1,2015-01-01,2015-01-01 07:55:00,5.0,Animal_Kingdom
2,2015-01-01,2015-01-01 08:05:00,5.0,Animal_Kingdom
3,2015-01-01,2015-01-01 08:10:00,10.0,Animal_Kingdom
4,2015-01-01,2015-01-01 08:20:00,10.0,Animal_Kingdom
...,...,...,...,...
149825,2018-12-31,2018-12-31 20:30:00,5.0,Animal_Kingdom
149826,2018-12-31,2018-12-31 20:40:00,5.0,Animal_Kingdom
149827,2018-12-31,2018-12-31 20:45:00,5.0,Animal_Kingdom
149828,2018-12-31,2018-12-31 20:50:00,5.0,Animal_Kingdom


,DATE,datetime,wait,park
0,2015-01-01,2015-01-01 07:45:00,5.0,Epcot
1,2015-01-01,2015-01-01 07:50:00,5.0,Epcot
2,2015-01-01,2015-01-01 08:40:00,5.0,Epcot
3,2015-01-01,2015-01-01 08:45:00,5.0,Epcot
4,2015-01-01,2015-01-01 08:50:00,5.0,Epcot
...,...,...,...,...
174039,2018-12-31,2018-12-31 23:30:00,15.0,Epcot
174040,2018-12-31,2018-12-31 23:35:00,15.0,Epcot
174041,2018-12-31,2018-12-31 23:45:00,15.0,Epcot
174042,2018-12-31,2018-12-31 23:50:00,15.0,Epcot


In [6]:
# Merge the two rides based on the date and time
df3=pd.merge(df1,df2, on='datetime',how='outer',suffixes=('_'+rides[0],'_'+rides[1]))
display(df3)

# Drop one of the date columns and rename the other to just DATE
df3=df3.drop(['DATE_dinosaur'],axis=1)
df3=df3.rename(columns={'DATE_expedition_everest':'DATE'})
display(df3)

# Merge the rides with the metadata
df=pd.merge(df3,meta,how='left',on='DATE')
display(df)

,DATE_dinosaur,datetime,wait_dinosaur,park_dinosaur,DATE_expedition_everest,wait_expedition_everest,park_expedition_everest
0,2015-01-01,2015-01-01 07:45:00,5.0,Animal_Kingdom,2015-01-01,5.0,Epcot
1,2015-01-01,2015-01-01 07:55:00,5.0,Animal_Kingdom,NaT,NaN,NaN
2,2015-01-01,2015-01-01 08:05:00,5.0,Animal_Kingdom,NaT,NaN,NaN
3,2015-01-01,2015-01-01 08:10:00,10.0,Animal_Kingdom,NaT,NaN,NaN
4,2015-01-01,2015-01-01 08:20:00,10.0,Animal_Kingdom,NaT,NaN,NaN
...,...,...,...,...,...,...,...
220745,NaT,2018-12-31 23:30:00,NaN,NaN,2018-12-31,15.0,Epcot
220746,NaT,2018-12-31 23:35:00,NaN,NaN,2018-12-31,15.0,Epcot
220747,NaT,2018-12-31 23:45:00,NaN,NaN,2018-12-31,15.0,Epcot
220748,NaT,2018-12-31 23:50:00,NaN,NaN,2018-12-31,15.0,Epcot


,datetime,wait_dinosaur,park_dinosaur,DATE,wait_expedition_everest,park_expedition_everest
0,2015-01-01 07:45:00,5.0,Animal_Kingdom,2015-01-01,5.0,Epcot
1,2015-01-01 07:55:00,5.0,Animal_Kingdom,NaT,NaN,NaN
2,2015-01-01 08:05:00,5.0,Animal_Kingdom,NaT,NaN,NaN
3,2015-01-01 08:10:00,10.0,Animal_Kingdom,NaT,NaN,NaN
4,2015-01-01 08:20:00,10.0,Animal_Kingdom,NaT,NaN,NaN
...,...,...,...,...,...,...
220745,2018-12-31 23:30:00,NaN,NaN,2018-12-31,15.0,Epcot
220746,2018-12-31 23:35:00,NaN,NaN,2018-12-31,15.0,Epcot
220747,2018-12-31 23:45:00,NaN,NaN,2018-12-31,15.0,Epcot
220748,2018-12-31 23:50:00,NaN,NaN,2018-12-31,15.0,Epcot


,datetime,wait_dinosaur,park_dinosaur,DATE,wait_expedition_everest,park_expedition_everest,DAYOFWEEK,DAYOFYEAR,WEEK,MONTH,YEAR,SEASON,DAYOFMONTH
0,2015-01-01 07:45:00,5.0,Animal_Kingdom,2015-01-01,5.0,Epcot,5.0,0.0,0.0,1.0,2015.0,CHRISTMAS PEAK,1.0
1,2015-01-01 07:55:00,5.0,Animal_Kingdom,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01 08:05:00,5.0,Animal_Kingdom,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01 08:10:00,10.0,Animal_Kingdom,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01 08:20:00,10.0,Animal_Kingdom,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220745,2018-12-31 23:30:00,NaN,NaN,2018-12-31,15.0,Epcot,2.0,364.0,52.0,12.0,2018.0,CHRISTMAS PEAK,31.0
220746,2018-12-31 23:35:00,NaN,NaN,2018-12-31,15.0,Epcot,2.0,364.0,52.0,12.0,2018.0,CHRISTMAS PEAK,31.0
220747,2018-12-31 23:45:00,NaN,NaN,2018-12-31,15.0,Epcot,2.0,364.0,52.0,12.0,2018.0,CHRISTMAS PEAK,31.0
220748,2018-12-31 23:50:00,NaN,NaN,2018-12-31,15.0,Epcot,2.0,364.0,52.0,12.0,2018.0,CHRISTMAS PEAK,31.0


In [7]:
# Write the final dataframe to a csv file
df.to_csv('wait_times.csv')